In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import re
import yaml
import numpy as np
import pandas as pd

import lts_functions as lts
import LTS_OSM

# Using jupyter notebooks with a virtual environment
# https://anbasile.github.io/posts/2017-06-25-jupyter-venv/
# https://stackoverflow.com/questions/58119823/jupyter-notebooks-in-visual-studio-code-does-not-use-the-active-virtual-environm
# ipython kernel install --user --name=.venv

In [6]:
# Settings
region = 'Cambridge'
unit = 'english'
tables = yaml.safe_load('config/tables.yml')

def read_yaml():
    with open('config/rating_dict.yml', 'r') as yml_file:
        rating_dict = yaml.safe_load(yml_file)
    return rating_dict

rating_dict = read_yaml()

In [3]:
# Load Data
gdfNodes, gdfEdges = LTS_OSM.download_data(region)

Loading saved graph for Cambridge
gdf_edges.shape=(105548, 193)
gdf_nodes.shape=(48501, 6)


In [4]:
def biking_permitted_yaml(gdf_edges, rating_dict):
    prefix = 'p'
    bikingPermittedRules = {k:v for (k,v) in rating_dict.items() if prefix in k}
    defaultRule = f'{prefix}0'

    gdf_edges['rule'] = defaultRule
    gdf_edges['biking_permitted'] = 'yes'

    for key, value in bikingPermittedRules.items():
        gdf_filter = gdf_edges.eval(value['condition'])
        gdf_edges.loc[gdf_filter, 'rule'] = key
        gdf_edges.loc[gdf_filter, 'biking_permitted'] = value['biking_permitted']

    return gdf_edges

gdf_biking_permitted = biking_permitted_yaml(gdfEdges.copy(), rating_dict)

In [7]:
def is_separated_path_yaml(gdf_edges, rating_dict):   
    prefix = 's'
    separatedPathRules = {k:v for (k,v) in rating_dict.items() if prefix in k}
    defaultRule = f'{prefix}0'

    gdf_edges['rule'] = defaultRule
    gdf_edges['bike_lane_separation'] = 'no'

    # get the columns that start with 'cycleway'
    # cycleway_tags = gdf_edges.columns[gdf_edges.columns.str.contains('cycleway')]

    for key, value in separatedPathRules.items():
        gdf_filter = gdf_edges.eval(value['condition'])
        gdf_edges.loc[gdf_filter, 'rule'] = key
        gdf_edges.loc[gdf_filter, 'bike_lane_separation'] = value['bike_lane_separation']

    return gdf_edges

gdf_separated_edges = is_separated_path_yaml(gdf_biking_permitted.copy(), rating_dict)

In [8]:
# Separated Edges
# separated_edges, unseparated_edges = lts.is_separated_path(gdf_allowed)
# print(f'{separated_edges.shape=}\n{unseparated_edges.shape=}')

# separated_edges_yaml, unseparated_edges_yaml = lts.is_separated_path_yaml(gdf_allowed, rating_dict)
separated_edges, unseparated_edges = lts.is_separated_path_yaml(gdf_allowed, rating_dict)
# print(f'{separated_edges_yaml.shape=}\n{unseparated_edges_yaml.shape=}')

# separated_edges['lts'] = 1

c:\Users\Scott\Documents\GitHub\BCU\StressMap\.venv\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
to_analyze, no_lane = lts.is_bike_lane(unseparated_edges)
parking_detected, parking_not_detected = lts.parking_present(to_analyze)

In [ ]:
def get_max_speed(gdf_edges):
    """
    Get the speed limit for ways
    If not available, make assumptions based on road type
    This errs on the high end of assumptions
    """
    pd.options.mode.chained_assignment = None  # default='warn'
    # create a list of conditions
    # When multiple conditions are satisfied, the first one encountered in conditions is used
    conditions = [
        (gdf_edges['maxspeed'] == 'national'),
        (gdf_edges['maxspeed'].isna()) & (gdf_edges['highway'] == 'motorway'),
        (gdf_edges['maxspeed'].isna()) & (gdf_edges['highway'] == 'primary'),
        (gdf_edges['maxspeed'].isna()) & (gdf_edges['highway'] == 'secondary'),
        (gdf_edges['maxspeed'].isna()),
        ]

    # create a list of the values we want to assign for each condition
    values = [national, motorway, primary, secondary, local]

    # create a new column and use np.select to assign values to it using our lists as arguments
    gdf_edges['maxspeed_assumed'] = np.select(conditions, values, default=gdf_edges['maxspeed'])

    # If mph
    if gdf_edges[gdf_edges['maxspeed_assumed'].astype(str).str.contains('mph')].shape[0] > 0:
        mph = gdf_edges['maxspeed_assumed'].str.contains('mph', na=False)
        gdf_edges['maxspeed_assumed'].loc[mph] = gdf_edges['maxspeed_assumed'][mph].str.split(
            ' ', expand=True)[0].apply(lambda x: np.array(x, dtype = 'int'))

    # if multiple speed values present, use the largest one
    gdf_edges['maxspeed_assumed'] = gdf_edges['maxspeed_assumed'].apply(
        lambda x: np.array(x, dtype = 'int')).apply(lambda x: np.max(x))

    return gdf_edges

In [ ]:
def get_max_speed_yaml(gdf_edges):



    # If mph
    if gdf_edges[gdf_edges['maxspeed_assumed'].astype(str).str.contains('mph')].shape[0] > 0:
        mph = gdf_edges['maxspeed_assumed'].str.contains('mph', na=False)
        gdf_edges['maxspeed_assumed'].loc[mph] = gdf_edges['maxspeed_assumed'][mph].str.split(
            ' ', expand=True)[0].apply(lambda x: np.array(x, dtype = 'int'))

    # if multiple speed values present, use the largest one
    gdf_edges['maxspeed_assumed'] = gdf_edges['maxspeed_assumed'].apply(
        lambda x: np.array(x, dtype = 'int')).apply(lambda x: np.max(x))
    
    return gdf_edges

In [ ]:
# Functions to update
def convert_feet_with_quotes(series):
    # Calculate decimal feet and inches when each given separately
    quoteValues = series.str.contains('\'')
    feetinch = series[quoteValues.fillna(False)].str.strip('"').str.split('\'', expand=True)
    feetinch = feetinch.apply(lambda x: np.array(x, dtype = 'int'))
    if feetinch.shape[0] > 0:
        feet = feetinch[0] + feetinch[1] / 12
        series[quoteValues.fillna(False)] = feet

    # Use larger value if given multiple
    multiWidth = series.str.contains(';')
    maxWidth = series[multiWidth.fillna(False)].str.split(';', expand=True).max(axis=1)
    series[multiWidth.fillna(False)] = maxWidth

    series = series.apply(lambda x: np.array(x, dtype = 'float'))

    return series

def bike_lane_analysis_with_parking(gdf_edges):
    # get lanes, width, speed
    gdf_edges = get_lanes(gdf_edges)
    gdf_edges = get_max_speed(gdf_edges)
    if unit == 'english':
        gdf_edges['width'] = convert_feet_with_quotes(gdf_edges['width'])

    # create a list of lts conditions
    # When multiple conditions are satisfied, the first one encountered in conditions is used
    conditions = [
        (gdf_edges['lanes_assumed'] >= 3) & (gdf_edges['maxspeed_assumed'] <= tables['s4'][unit]),
        (gdf_edges['width'] <= tables['w2'][unit]),
        (gdf_edges['width'] <= tables['w3'][unit]),
        (gdf_edges['width'] <= tables['w4'][unit]) &
            ((gdf_edges['maxspeed_assumed'] <= tables['s2'][unit]) &
                (gdf_edges['highway'] == 'residential')),
        (gdf_edges['maxspeed_assumed'] > tables['s2'][unit]) &
            (gdf_edges['maxspeed_assumed'] <= tables['s3'][unit]),
        (gdf_edges['maxspeed_assumed'] > tables['s3'][unit]) & 
            (gdf_edges['maxspeed_assumed'] <= tables['s4'][unit]),
        (gdf_edges['maxspeed_assumed'] > tables['s4'][unit]),
        (gdf_edges['highway'] != 'residential')
        ]

    # create a list of the values we want to assign for each condition
    values = ['b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
    gdf_edges['rule'] = np.select(conditions, values, default='b1')
    rule_dict = {'b1':1, 'b2':3, 'b3':3, 'b4':2, 'b5':2, 'b6':2, 'b7':3, 'b8':4, 'b9':3}
    gdf_edges['lts'] = gdf_edges['rule'].map(rule_dict)

    return gdf_edges

def bike_lane_analysis_no_parking(gdf_edges):
    """
    LTS depends on presence of median, but this is not commonly tagged in OSM. 
    Possibly check the 'dual_carriageway' tag. 
    """

    # get lanes, width, speed
    gdf_edges = get_lanes(gdf_edges)
    gdf_edges = get_max_speed(gdf_edges)
    if unit == 'english':
        gdf_edges['width'] = convert_feet_with_quotes(gdf_edges['width'])

    # assign widths that are a string to nan
    gdf_edges.loc[gdf_edges[['width']].map( # applymap depreciated
        lambda x: isinstance(x, str))['width'], 'width'] = np.nan

    # create a list of lts conditions
    # When multiple conditions are satisfied, the first one encountered in conditions is used
    conditions = [
        (gdf_edges['lanes_assumed'] >= 3) & (gdf_edges['maxspeed_assumed'] <= tables['s5'][unit]),
        (gdf_edges[['width']].map(lambda x: isinstance(x, float))['width']) & # applymap depreciated
            (gdf_edges['width'] <= tables['w1'][unit]),
        (gdf_edges['maxspeed_assumed'] > tables['s3'][unit]) &
            (gdf_edges['maxspeed_assumed'] <= tables['s5'][unit]),
        (gdf_edges['maxspeed_assumed'] > tables['s5'][unit]),
        (gdf_edges['highway'] != 'residential')
        ]

    values = ['c3', 'c4', 'c5', 'c6', 'c7']
    gdf_edges['rule'] = np.select(conditions, values, default='c1')
    rule_dict = {'c1':1, 'c3':3, 'c4':2, 'c5':3, 'c6':4, 'c7':3}
    gdf_edges['lts'] = gdf_edges['rule'].map(rule_dict)

    return gdf_edges

def mixed_traffic(gdf_edges):
    # get lanes, width, speed
    gdf_edges = get_lanes(gdf_edges)
    gdf_edges = get_max_speed(gdf_edges)

    # create a list of lts conditions
    # When multiple conditions are satisfied, the first one encountered in conditions is used
    conditions = [
        (gdf_edges['motor_vehicle'] == 'no'),
        (gdf_edges['highway'] == 'pedestrian'),
        (gdf_edges['highway'] == 'footway') & (gdf_edges['footway'] == 'crossing'),
        (gdf_edges['highway'] == 'service') & (gdf_edges['service'] == 'alley'),
        (gdf_edges['highway'] == 'track'),
        (gdf_edges['maxspeed_assumed'] <= tables['s3'][unit]) & (gdf_edges['highway'] == 'service') &
            (gdf_edges['service'] == 'parking_aisle'),
        (gdf_edges['maxspeed_assumed'] <= tables['s3'][unit]) & (gdf_edges['highway'] == 'service') &
            (gdf_edges['service'] == 'driveway'),
        (gdf_edges['maxspeed_assumed'] <= tables['s1'][unit]) & (gdf_edges['highway'] == 'service'),
        (gdf_edges['maxspeed_assumed'] <= tables['s2'][unit]) & (gdf_edges['lanes_assumed'] <= 3) &
            (gdf_edges['highway'] == 'residential'),
        (gdf_edges['maxspeed_assumed'] <= tables['s2'][unit]) & (gdf_edges['lanes_assumed'] <= 3),
        (gdf_edges['maxspeed_assumed'] <= tables['s2'][unit]) & (gdf_edges['lanes_assumed'] <= 5),
        (gdf_edges['maxspeed_assumed'] <= tables['s2'][unit]) & (gdf_edges['lanes_assumed'] > 5),
        (gdf_edges['maxspeed_assumed'] <= tables['s3'][unit]) & (gdf_edges['lanes_assumed'] < 3) &
            (gdf_edges['highway'] == 'residential'),
        (gdf_edges['maxspeed_assumed'] <= tables['s3'][unit]) & (gdf_edges['lanes_assumed'] <= 3),
        (gdf_edges['maxspeed_assumed'] <= tables['s3'][unit]) & (gdf_edges['lanes_assumed'] > 3),
        (gdf_edges['maxspeed_assumed'] > tables['s3'][unit])
        ]

    # create a list of the values we want to assign for each condition
    values = ['m17', 'm13', 'm14', 'm2', 'm15', 'm3', 
              'm4', 'm16', 'm5', 'm6', 'm7',
              'm8', 'm9', 'm10', 'm11', 'm12']

    # create a new column and use np.select to assign values to it using our lists as arguments
    gdf_edges['rule'] = np.select(conditions, values, default='m0')

    rule_dict = {'m17':1, 'm13':1, 'm14':2, 'm2':2, 'm15':2, 'm3':2, 'm4':2, 'm16':2,
                 'm5':2, 'm6':3, 'm7':3, 'm8':4, 'm9':2, 'm10':3, 'm11':4, 'm12':4}

    gdf_edges['lts'] = gdf_edges['rule'].map(rule_dict)

    return gdf_edges

In [ ]:
if parking_detected.shape[0] > 0:
    parking_lts = lts.bike_lane_analysis_with_parking(parking_detected)
else:
    parking_lts = parking_detected

if parking_not_detected.shape[0] > 0:
    no_parking_lts = lts.bike_lane_analysis_no_parking(parking_not_detected)
else:
    no_parking_lts = parking_not_detected

In [ ]:
testCondition = 'p7'

queryStr = rating_dict[testCondition]['condition']

filterYaml = gdfEdges.query(queryStr)

filterFunc = gdf_not_allowed[gdf_not_allowed['rule'] == testCondition]

print(queryStr)
print(f'{filterFunc.shape=}\n{filterYaml.shape=}')